In [1]:
import torch
torch.set_num_threads(16)
torch.set_num_interop_threads(16)

In [2]:
import cornac
from cornac.eval_methods import RatioSplit
from cornac.models import MF, PMF, BPR, SANSA, BiVAECF, LightGCN, RecVAE, EASE, NGCF, VAECF, IBPR, NeuMF, HPF, GCMC, WBPR
from cornac.metrics import Precision, Recall, NDCG, MAP, MRR

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

/Users/a-shyraliev/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Import datasets

In [3]:
from typing import Optional

def stratified_ranking_split(
    df: pd.DataFrame,
    entity_field: str,
    test_size: float = 0.1,
    random_state: Optional[int] = None,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Splits a ranking-based dataset into training and test sets while preserving the entity distribution.

    This function is useful for ranking models (e.g., next-best-offer, personalized recommendations) where each
    entity (e.g., user) has multiple interactions with different items, and stratification ensures that
    different user interaction levels are maintained in both splits.

    Parameters:
        df: The ranking dataset, where each row represents an interaction between an entity (e.g., user)
            and an item (e.g., game, offer).
        entity_field: The column representing the entity to be stratified.
        test_size: Fraction of unique entities to allocate to the test set.
        random_state: Random seed for reproducibility.

    Returns:
        DataFrames containing training and test data.

    Example:
        >>> train_validation_df, test_df = stratified_ranking_split(df, entity_field='user_id', test_size=0.1)
        >>> train_df, validation_df = stratified_ranking_split(
        ...     train_validation_df, entity_field='user_id', test_size=0.1
        ... )
        >>> print(train_df.shape, validation_df.shape, test_df.shape)
    """
    entity_interaction_counts = df[entity_field].value_counts()

    interaction_frequencies = entity_interaction_counts.value_counts()
    stratifiable_interaction_counts = interaction_frequencies[interaction_frequencies >= 2].index

    stratifiable_entities = entity_interaction_counts[
        entity_interaction_counts.isin(stratifiable_interaction_counts)
    ].index
    non_stratifiable_entities = entity_interaction_counts[
        ~entity_interaction_counts.isin(stratifiable_interaction_counts)
    ].index

    train_strat, test_strat = (
        train_test_split(
            stratifiable_entities,
            test_size=test_size,
            stratify=entity_interaction_counts[stratifiable_entities],
            random_state=random_state,
        )
        if len(stratifiable_entities) > 1
        else (stratifiable_entities, [])
    )

    if len(non_stratifiable_entities) > 1:
        train_non_strat, test_non_strat = train_test_split(
            non_stratifiable_entities,
            test_size=test_size,
            random_state=random_state,
        )
    else:
        train_non_strat = non_stratifiable_entities
        test_non_strat = []

    train_users = np.concatenate([train_strat, train_non_strat])
    test_users = np.concatenate([test_strat, test_non_strat])

    return df[df[entity_field].isin(train_users)], df[df[entity_field].isin(test_users)]

In [4]:
SEED = 123

metrics = [Precision(k=10), Recall(k=10), NDCG(k=10), MAP(), MRR()]

In [5]:
steam_dataset = (
    pd.read_csv(
        "/Users/a-shyraliev/phd/rec-sys-research/collab_filtering_battlefield/steam_recommendations.csv",
        usecols=['user_id', 'app_id', 'hours'],
    )
    .loc[:, ['user_id', 'app_id', 'hours']]
    .drop_duplicates()
)

In [6]:
steam_dataset['user_id'].nunique(), steam_dataset['app_id'].nunique()

(13781059, 37610)

In [7]:
_, steam_dataset_sample_str = stratified_ranking_split(
    steam_dataset,
    entity_field='user_id',
    test_size=0.01,
    random_state=SEED,
)

del steam_dataset, _

In [8]:
rs = RatioSplit(data=steam_dataset_sample_str.values, test_size=0.2, rating_threshold=0.0, seed=SEED)
rs.train_set.csr_matrix, rs.test_set.csr_matrix

(<121521x14807 sparse matrix of type '<class 'numpy.float64'>'
 	with 325828 stored elements in Compressed Sparse Row format>,
 <121521x14807 sparse matrix of type '<class 'numpy.float64'>'
 	with 62707 stored elements in Compressed Sparse Row format>)

### 1. MF hyperparams optimization

In [9]:
mf_models = [
    MF(name='MF k=10, lambda_reg=0.02', k=10, max_iter=200, lambda_reg=0.02, use_bias=True, seed=SEED),
]

In [10]:
cornac.Experiment(
    eval_method=rs,
    models=mf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                         | MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------------------------ + --- + ------ + ------- + ------------ + --------- + --------- + --------
MF k=10, lambda_reg=0.02 | nan | 0.0005 |  0.0001 |       0.0000 |    0.0003 |    0.9874 |  23.3163



### 2. PMF hyperparams optimization

In [11]:
pmf_models = [
    PMF(name='PMF k=5, lambda_reg=0.001', k=5, max_iter=200, lambda_reg=0.001, seed=SEED),
]

In [12]:
cornac.Experiment(
    eval_method=rs,
    models=pmf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                          |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------------------------- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
PMF k=5, lambda_reg=0.001 | 0.0019 | 0.0024 |  0.0019 |       0.0005 |    0.0039 |    5.6411 |  59.3444



### 3. BPR hyperparams optimization

In [13]:
bpr_models = [
    BPR(name='BPR k=50, lambda_reg=0.01', k=50, max_iter=200, lambda_reg=0.01, seed=SEED),
]

In [14]:
cornac.Experiment(
    eval_method=rs,
    models=bpr_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                          |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------------------------- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
BPR k=50, lambda_reg=0.01 | 0.0310 | 0.0425 |  0.0318 |       0.0094 |    0.0578 |    7.3132 |  61.8743



### 4. BiVAECF hyperparams optimization

In [15]:
bivaecf_models = [
    BiVAECF(name='BiVAECF k=10, encoder_structure=[20]', k=10, encoder_structure=[20], n_epochs=10, use_gpu=False, seed=SEED),
]

In [16]:
cornac.Experiment(
    eval_method=rs,
    models=bivaecf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                                     |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------------------------------------ + ------ + ------ + ------- + ------------ + --------- + --------- + --------
BiVAECF k=10, encoder_structure=[20] | 0.0167 | 0.0223 |  0.0151 |       0.0044 |    0.0279 |  403.8871 |  52.1578



### 5. RecVAE hyperparams optimization

In [43]:
recvae_models = [
    RecVAE(name='RecVAE hidden_dim=600, latent_dim=200', hidden_dim=600, latent_dim=200, n_epochs=10, use_gpu=False, seed=SEED),
]

In [44]:
cornac.Experiment(
    eval_method=rs,
    models=recvae_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                                      |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------------------------------------- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
RecVAE hidden_dim=600, latent_dim=200 | 0.0304 | 0.0463 |  0.0339 |       0.0107 |    0.0585 | 4152.2937 | 118.4998



### 6. EASE hyperparams optimization

In [17]:
ease_models = [
    EASE(name='EASE lamb=800, posB=True', lamb=800, posB=True, seed=SEED),
]

In [18]:
cornac.Experiment(
    eval_method=rs,
    models=ease_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                         |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------------------------ + ------ + ------ + ------- + ------------ + --------- + --------- + --------
EASE lamb=800, posB=True | 0.0408 | 0.0577 |  0.0461 |       0.0126 |    0.0771 |   31.2813 |  44.3687



### 7. HPF hyperparams optimization

In [19]:
hpf_models = [
    HPF(name='HPF k=5, hierarchical=False', k=5, hierarchical=False, seed=SEED),
]

In [20]:
cornac.Experiment(
    eval_method=rs,
    models=hpf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

Learning...
Learning completed!

TEST:
...
                            |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
--------------------------- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
HPF k=5, hierarchical=False | 0.0198 | 0.0286 |  0.0214 |       0.0065 |    0.0389 |   53.2573 |  61.7736



### 8. IBPR hyperparams optimization

In [49]:
ibpr_models = [
    IBPR(name='IBPR k=20, lamda=0.001', k=20, lamda=0.001),
]

In [ ]:
cornac.Experiment(
    eval_method=rs,
    models=ibpr_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

In [ ]:
## too long to train
472m of training time - still not converged

### 9. SANSA hyperparams optimization

In [45]:
sansa_models = [
    # SANSA(name='SANSA l2=1, factorizer_class=CHOLMOD', l2=1, factorizer_class="CHOLMOD", verbose=False, seed=SEED),
    # SANSA(name='SANSA l2=10, factorizer_class=CHOLMOD', l2=10, factorizer_class="CHOLMOD", verbose=False, seed=SEED),
    # SANSA(name='SANSA l2=500, factorizer_class=CHOLMOD', l2=500, factorizer_class="CHOLMOD", verbose=False, seed=SEED),
    SANSA(name='SANSA l2=1, factorizer_class=ICF', l2=1, factorizer_class="ICF", verbose=False, seed=SEED),
    SANSA(name='SANSA l2=10, factorizer_class=ICF', l2=10, factorizer_class="ICF", verbose=False, seed=SEED),
    SANSA(name='SANSA l2=500, factorizer_class=ICF', l2=500, factorizer_class="ICF", verbose=False, seed=SEED),
]

In [46]:
cornac.Experiment(
    eval_method=rs,
    models=sansa_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

INFO:sansa.core.factorizers:Computing incomplete Cholesky decomposition of X^TX + 1*I...
INFO:sansa.core.factorizers:Finding a fill-in reducing ordering (method = colamd)...
INFO:sansa.core.factorizers:Computing X^TX...
INFO:sansa.core.factorizers:
                X^TX info:
                    shape = (14807, 14807) 
                    nnz = 2905784 
                    density = 1.325346% 
                    size = 23.3 MB
                
INFO:sansa.core.factorizers:Sorting indices of A...
INFO:sansa.core.factorizers:Casting indptr of A to int64...
INFO:sansa.core.factorizers:Casting indices of A to int64...
INFO:sansa.core.factorizers:Computing approximate Cholesky decomposition (method = ICF)...
INFO:sansa.core._ops._factor_ops:
                Incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A 
                with L2=1.0000e+00. Continuing with L2=1.0010e+00.
                
INFO:sansa.core._ops._factor_ops:
                Incomple


TEST:
...
                                   |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
---------------------------------- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
SANSA l2=1, factorizer_class=ICF   | 0.0191 | 0.0300 |  0.0225 |       0.0066 |    0.0375 |   16.8820 |  36.4241
SANSA l2=10, factorizer_class=ICF  | 0.0191 | 0.0300 |  0.0225 |       0.0066 |    0.0375 |   16.0139 |  35.5143
SANSA l2=500, factorizer_class=ICF | 0.0191 | 0.0300 |  0.0226 |       0.0066 |    0.0376 |    6.6424 |  33.9904



### 10. VAECF hyperparams optimization

In [47]:
vaecf_models = [
    VAECF(name='VAECF k=5, likelihood=mult', k=5, likelihood="mult", n_epochs=10, use_gpu=False, seed=SEED),
    VAECF(name='VAECF k=5, likelihood=gaus', k=5, likelihood="gaus", n_epochs=10, use_gpu=False, seed=SEED),
    VAECF(name='VAECF k=10, likelihood=mult', k=10, likelihood="mult", n_epochs=10, use_gpu=False, seed=SEED),
    VAECF(name='VAECF k=10, likelihood=gaus', k=10, likelihood="gaus", n_epochs=10, use_gpu=False, seed=SEED),
]

In [48]:
cornac.Experiment(
    eval_method=rs,
    models=vaecf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                            |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) |  Test (s)
--------------------------- + ------ + ------ + ------- + ------------ + --------- + --------- + ---------
VAECF k=5, likelihood=mult  | 0.0298 | 0.0410 |  0.0316 |       0.0100 |    0.0614 |  297.3855 |   58.0917
VAECF k=5, likelihood=gaus  | 0.0268 | 0.0354 |  0.0263 |       0.0079 |    0.0511 | 3190.6507 | 1090.2030
VAECF k=10, likelihood=mult | 0.0290 | 0.0396 |  0.0300 |       0.0095 |    0.0582 | 2086.4787 |   57.7111
VAECF k=10, likelihood=gaus | 0.0269 | 0.0356 |  0.0264 |       0.0079 |    0.0511 | 1260.1899 |   56.8296



### 11. NeuMF hyperparams optimization

In [11]:
neumf_models = [
    NeuMF(name='NeuMF num_factors=8', num_factors=8, verbose=False, backend="pytorch", seed=SEED),
    NeuMF(name='NeuMF num_factors=20', num_factors=20, verbose=False, backend="pytorch", seed=SEED),
]

In [12]:
cornac.Experiment(
    eval_method=rs,
    models=neumf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                     |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
-------------------- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
NeuMF num_factors=8  | 0.0270 | 0.0392 |  0.0282 |       0.0089 |    0.0498 |  303.9657 | 121.3155
NeuMF num_factors=20 | 0.0268 | 0.0388 |  0.0282 |       0.0089 |    0.0510 |  302.2828 | 122.5067



### 12. LightGCN hyperparams optimization

In [9]:
lightgcn_models = [
    LightGCN(verbose=False, num_epochs=10, seed=SEED),
]

In [10]:
cornac.Experiment(
    eval_method=rs,
    models=lightgcn_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

/Users/a-shyraliev/Library/Python/3.9/lib/python/site-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


KeyboardInterrupt: 

1125 mins of training - not finished

### 13. NGCF hyperparams optimization

In [ ]:
ngcf_models = [
    NGCF(verbose=False, num_epochs=10, seed=SEED),
]

In [ ]:
cornac.Experiment(
    eval_method=rs,
    models=ngcf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

it's slower than LightGCN -> won't finish

### 14. GCMC hyperparams optimization

In [10]:
gcmc_models = [
    GCMC(verbose=False, max_iter=100, seed=SEED),
]

In [11]:
cornac.Experiment(
    eval_method=rs,
    models=gcmc_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

/Users/a-shyraliev/Library/Python/3.9/lib/python/site-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


AssertionError: 

### 15. WBPR hyperparams optimization

In [9]:
wbpr_models = [
    WBPR(seed=SEED),
]

In [10]:
cornac.Experiment(
    eval_method=rs,
    models=wbpr_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
     |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
---- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
WBPR | 0.0051 | 0.0080 |  0.0030 |       0.0009 |    0.0048 |    1.9612 |  55.3430

